In [4]:
#https://www.kaggle.com/datasets/msambare/fer2013/data
##https://medium.com/analytics-vidhya/implementing-svm-for-performing-classification-and-finding-accuracy-in-python-using-datasets-wine-e4fef8e804b4
#https://towardsdatascience.com/performance-metrics-confusion-matrix-precision-recall-and-f1-score-a8fe076a2262
#https://medium.com/swlh/confusion-matrix-and-classification-report-88105288d48f
#https://numpy.org/doc/stable/reference/generated/numpy.fromstring.html
#https://neptune.ai/blog/how-to-use-google-colab-for-deep-learning-complete-tutorial
#https://scikit-learn.org/stable/modules/model_evaluation.html
#https://www.tensorflow.org/tutorials/images/cnn
#https://stackoverflow.com/questions/53609697/keras-what-accuracy-metric-should-be-used-along-with-sparse-categorical-crosse

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.utils import to_categorical
'''
data = pd.read_csv("fer2013.csv")

#image 7
image_row = data.iloc[7]

#pixel values and reshape the image into a square
pixels = image_row['pixels'].split()
image_array = np.array(pixels, dtype='uint8').reshape((48, 48))

#show image
plt.imshow(image_array, cmap='gray')
plt.axis('off')
plt.show()
'''


'\ndata = pd.read_csv("fer2013.csv")\n\n#image 7\nimage_row = data.iloc[7]\n\n#pixel values and reshape the image into a square\npixels = image_row[\'pixels\'].split()\nimage_array = np.array(pixels, dtype=\'uint8\').reshape((48, 48))\n\n#show image\nplt.imshow(image_array, cmap=\'gray\')\nplt.axis(\'off\')\nplt.show()\n'

In [7]:
#PART A1
#CSV file into DataFrame data
data = pd.read_csv('fer2013.csv')

#Splitting the DataFrame from "Usage" column
train_data = data[data['Usage'] == 'Training']
test_data = data[data['Usage'] == 'PrivateTest']

#extracting the pixels and labels for training and test sets
X_train = np.array([np.fromstring(pixel, dtype=int, sep=' ') for pixel in train_data['pixels']])
y_train = train_data['emotion'].values

X_test = np.array([np.fromstring(pixel, dtype=int, sep=' ') for pixel in test_data['pixels']])
y_test = test_data['emotion'].values

#SVM classifier
svm_classifier = SVC()

#raining the SVM classifier
svm_classifier.fit(X_train, y_train)

#test set
y_pred = svm_classifier.predict(X_test)

#evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

#evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

#confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)


Accuracy: 0.4471997770966843
Precision: 0.45357166337232335
Recall: 0.4471997770966843
F1 Score: 0.42950209418514934
Confusion Matrix:
[[109   0  46 142  96  20  78]
 [  7   3   3  24   6   2  10]
 [ 39   0 122 131 111  50  75]
 [ 25   0  30 636 101  18  69]
 [ 35   0  50 136 241  11 121]
 [ 13   0  40  68  31 219  45]
 [ 28   0  25 182 101  15 275]]


In [8]:
#PART B1
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.utils import to_categorical

#Splitting validation
val_data = data[data['Usage'] == 'PublicTest']
X_val = np.array([np.fromstring(pixel, dtype=int, sep=' ') for pixel in val_data['pixels']])

y_train = to_categorical(train_data['emotion'], num_classes=7)
y_val = to_categorical(val_data['emotion'], num_classes=7)
y_test = test_data['emotion'].values

#reshape the data into 48x48 grayscale images
X_train = X_train.reshape(-1, 48, 48, 1)
X_val = X_val.reshape(-1, 48, 48, 1)
X_test = X_test.reshape(-1, 48, 48, 1)

#normalize pixel values to be between 0 and 1
X_train = X_train.astype('float32') / 255
X_val = X_val.astype('float32') / 255
X_test = X_test.astype('float32') / 255
#CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

#Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=30, batch_size=128, validation_data=(X_val, y_val))

y_pred = np.argmax(model.predict(X_test), axis=-1)

#evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)


Epoch 1/30
225/225 [==============================] - 113s 490ms/step - loss: 1.7676 - accuracy: 0.3473 - val_loss: 1.9504 - val_accuracy: 0.2003
Epoch 2/30
225/225 [==============================] - 118s 525ms/step - loss: 1.4309 - accuracy: 0.4500 - val_loss: 1.8987 - val_accuracy: 0.2421
Epoch 3/30
225/225 [==============================] - 106s 470ms/step - loss: 1.3053 - accuracy: 0.5040 - val_loss: 1.3224 - val_accuracy: 0.5029
Epoch 4/30
225/225 [==============================] - 116s 516ms/step - loss: 1.2138 - accuracy: 0.5404 - val_loss: 1.2323 - val_accuracy: 0.5322
Epoch 5/30
225/225 [==============================] - 110s 489ms/step - loss: 1.1377 - accuracy: 0.5669 - val_loss: 1.2022 - val_accuracy: 0.5497
Epoch 6/30
225/225 [==============================] - 120s 535ms/step - loss: 1.0577 - accuracy: 0.6026 - val_loss: 1.2804 - val_accuracy: 0.5252
Epoch 7/30
225/225 [==============================] - 106s 471ms/step - loss: 0.9926 - accuracy: 0.6237 - val_loss: 1.2243 -

In [24]:
#PART A2
from sklearn.model_selection import cross_val_score, KFold
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, classification_report

# phoebe data
data = pd.read_csv('phoebe_AU.csv')

#Action Units
X = data.drop(['file_name', 'label'], axis=1)
y = data['label']

#convert 'unknown' label to a numerical value, for example, 0
#0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral

y = y.replace({'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'sad': 4, 'surprise': 5, 'unknown': 6})
svm_model = make_pipeline(StandardScaler(), SVC(kernel='linear'))

#5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
accuracy_scores = cross_val_score(svm_model, X, y, cv=kf, scoring='accuracy')

print("Accuracy scores for 5-fold cross-validation:")
print(accuracy_scores)
print("Mean accuracy:", accuracy_scores.mean())

#svm
svm_model.fit(X, y)
y_pred = svm_model.predict(X)

#evaluation metrics
conf_matrix = confusion_matrix(y, y_pred)
classification_rep = classification_report(y, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_rep)


Accuracy scores for 5-fold cross-validation:
[0.3        0.35       0.3        0.31578947 0.31578947]
Mean accuracy: 0.3163157894736842

Confusion Matrix:
[[11  0  0  0  0  0]
 [ 0  9  0  0  0  0]
 [ 0  0 33  0  0  0]
 [ 0  0  0 16  1  0]
 [ 0  0  0  0 16  0]
 [ 0  0  0  0  0 12]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       1.00      1.00      1.00         9
           3       1.00      1.00      1.00        33
           4       1.00      0.94      0.97        17
           5       0.94      1.00      0.97        16
           6       1.00      1.00      1.00        12

    accuracy                           0.99        98
   macro avg       0.99      0.99      0.99        98
weighted avg       0.99      0.99      0.99        98



In [22]:
# PART B2
import numpy as np
import os
from keras.preprocessing import image

#Phoebe "unknown" images
unknown_images_path = "/content/drive/MyDrive/Colab Notebooks/images/unknown"
unknown_image_files = os.listdir("/content/drive/MyDrive/Colab Notebooks/images/unknown")

#preprocess the images
unknown_images = []
for image_file in unknown_image_files:
    img_path = os.path.join(unknown_images_path, image_file)
    img = image.load_img(img_path, target_size=(48, 48), grayscale=True)
    img_array = image.img_to_array(img)
    img_array /= 255.0  # Normalize pixel values
    unknown_images.append(img_array)

unknown_images = np.array(unknown_images)
unknown_predictions = model.predict(unknown_images)
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
predicted_labels = [emotion_labels[np.argmax(prediction)] for prediction in unknown_predictions]

#show labels
for image_file, predicted_label in zip(unknown_image_files, predicted_labels):
    print(f"{image_file}: {predicted_label}")


/usr/local/lib/python3.10/dist-packages/keras/src/utils/image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(


1/1 [==============================] - 0s 45ms/step
41_06.jpg: happy
1_01.jpg: angry
48_01.jpg: happy
46_03.jpg: happy
35_42.jpg: sad
8_01.jpg: happy
4_01.jpg: sad
4_20.jpg: sad
9_41.jpg: happy
26_123.jpg: happy
44_01.jpg: neutral
52_31.jpg: sad


In [27]:
import os
import numpy as np
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.optimizers import Adam

#Phoebe "unknown" images
unknown_images_path = "/content/drive/MyDrive/Colab Notebooks/images/unknown"

#CNN model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

model.compile(optimizer=Adam(lr=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#trainin the model on Phoebe expressions image dataset
history = model.fit(X_train, y_train,
                    epochs=30,
                    batch_size=128,
                    validation_data=(X_val, y_val))

#loading Phoebe "unknown" images
unknown_image_files = os.listdir(unknown_images_path)

#preprocess the images
unknown_images = []
for image_file in unknown_image_files:
    img_path = os.path.join(unknown_images_path, image_file)
    img = image.load_img(img_path, target_size=(48, 48), grayscale=True)
    img_array = image.img_to_array(img)
    img_array /= 255.0  # Normalize pixel values
    unknown_images.append(img_array)

unknown_images = np.array(unknown_images)

#trained model
unknown_predictions = model.predict(unknown_images)
#emotion labels
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
predicted_labels = [emotion_labels[np.argmax(prediction)] for prediction in unknown_predictions]
#predicted labels
for image_file, predicted_label in zip(unknown_image_files, predicted_labels):
    print(f"{image_file}: {predicted_label}")



Epoch 1/30
225/225 [==============================] - 118s 510ms/step - loss: 1.7517 - accuracy: 0.3526 - val_loss: 1.8985 - val_accuracy: 0.2533
Epoch 2/30
225/225 [==============================] - 135s 602ms/step - loss: 1.4348 - accuracy: 0.4531 - val_loss: 1.5511 - val_accuracy: 0.3826
Epoch 3/30
225/225 [==============================] - 116s 517ms/step - loss: 1.3059 - accuracy: 0.5075 - val_loss: 1.4830 - val_accuracy: 0.4430
Epoch 4/30
225/225 [==============================] - 121s 538ms/step - loss: 1.2085 - accuracy: 0.5456 - val_loss: 1.2536 - val_accuracy: 0.5291
Epoch 5/30
225/225 [==============================] - 132s 590ms/step - loss: 1.1351 - accuracy: 0.5732 - val_loss: 1.2739 - val_accuracy: 0.5263
Epoch 6/30
225/225 [==============================] - 127s 562ms/step - loss: 1.0579 - accuracy: 0.6021 - val_loss: 1.3311 - val_accuracy: 0.5127
Epoch 7/30
225/225 [==============================] - 137s 608ms/step - loss: 0.9822 - accuracy: 0.6292 - val_loss: 1.2252 -

In [ ]:
'''
SVM-Fer2013 -Uses Fer2013 dataset with SVM
SVM-OpenFace -Uses Phoebe AU.csv with SVM
NN-Fer2013 -Uses Fer2013 dataset with Neural Network
NN-FineTuned -Usies Fine-Tuned Neural Network on Phoebe-face images

SVM-OpenFace worked best with this dataset image 4_20.jpg represents sad emotion expressions